In [1]:
import logging, os
from gcs_helpers import add_record_ids, sheets_upload, format_linked_records
import pandas as pd # To work with data
from pyairtable import Base, metadata # To Access Airtable 
from dotenv import load_dotenv # To Access Environment Variables

# Declare Variables from .env
load_dotenv()
AIRTABLE_API_KEY = os.environ.get('AIRTABLE_API_KEY')
AIRTABLE_BASE_ID = os.environ.get('AIRTABLE_BASE_ID')
directories = ['Tables', 'Staging','JSON']

# Declare primary fact table
fact_tables = ['Movies']

# Load Base
my_base = Base(AIRTABLE_API_KEY, AIRTABLE_BASE_ID)
my_base_metadata = metadata.get_base_schema(my_base)['tables']
schema = metadata.get_base_schema(my_base)
tables = schema['tables']
table_ids = [x['id'] for x in tables]

# For each table in a base
for table in tables:

    # Get Table Data and create json from output
    table_name = table['name']

    print(f'Getting data for Table: {table_name}')

    table_data_raw = my_base.all(table['id'])
    table_data = add_record_ids(table_data_raw) #Add airtable record ids to output
    table_df = pd.DataFrame(table_data)

    # Get Array Fields
    table_index = tables.index(table)
    table_fields = my_base_metadata[table_index]['fields']
    linked_fields = [x for x in table_fields if x['type'] in 'multipleRecordLinks']
    array_field_types = ['multipleRecordLinks','multipleCollaborators','multipleSelects','multipleAttachments','mutipleLookupValues']
    array_fields = [x['name'] for x in table_fields if x['type'] in array_field_types]
    non_array_fields = [x['name'] for x in table_fields if x['type'] not in array_field_types and table_fields.index(x) != 0]

    # If table is the primary fact table - explode linked record columns and create a join.json file
    if table_name in fact_tables:

        print('Formatting Fact Table')
        # Format Linked Records
        table_df = format_linked_records(linked_fields,table_df,tables,table_ids,my_base)
        
        # Transform Join Table by exploding fields
        join_df = table_df
        join_df = join_df.drop(columns=non_array_fields)
        for field in array_fields:
            join_df[field] = join_df[field].apply(lambda d: d if isinstance(d, list) else [])
            join_df = join_df.explode(field)
        
        # Upload tables to google sheets
        join_table_name = f'{table_name} JOIN'
        sheets_upload(join_df, join_table_name)
        sheets_upload(table_df, table_name)
    else:
        #Drop Array Fields that are captured in join_df
        table_df = table_df.drop(columns=array_fields)
        sheets_upload(table_df,table_name)

print('Success! All Tables Uploaded to Google Sheet')


Getting data for Table: Movies
Formatting Fact Table
Formatting Linked Records for genres_linked field
Formatting Linked Records for collection field
No sheet found for Movies JOIN, creating new table
Successfully uploaded Movies JOIN! 

Sheet for Movies found. appending table
Successfully uploaded Movies! 

Getting data for Table: Genre
Sheet for Genre found. appending table
Successfully uploaded Genre! 

Getting data for Table: Collections
Sheet for Collections found. appending table
Successfully uploaded Collections! 

Success! All Tables Uploaded to Google Sheet


In [2]:
table_df.columns

Index(['collection', 'id', 'airtable_id'], dtype='object')